# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
### Import all dependencies

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import csv

### Scrap postal code with other related data using BeautifulSoup library and import it in pandas dataframe

In [5]:
post_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
posts = requests.get(post_url)
parser = BeautifulSoup(posts.content,"html.parser")
table = parser.find_all('table')[0] 
df = pd.read_html(str(table),header=0)[0]
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Remove rows where Borough contains '_Not assigned_' values

In [6]:
df = df[~df['Borough'].isin(['Not assigned'])]
df.reset_index(inplace=True,drop=True)
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### More than one neighborhood can exist in one postal code area

In [7]:
df['Postcode'].nunique()

103

### Combile all values by concate with same postal code

In [8]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### 9th cell in the table on the Wikipedia page

In [9]:
df['Neighbourhood'] = df['Borough'].where(df['Neighbourhood']=='Not assigned',df['Neighbourhood'])
df.loc[df['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


### Total no of rows in current dataframe

In [10]:
df.shape

(103, 3)

### Import Geodata into dataframe

In [11]:
geocode = pd.read_csv('Geospatial_Coordinates.csv')
geocode.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

### Final Dataframe with geodata

In [12]:
com_df = df.merge(geocode, how='left', on="Postcode" )
com_df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [16]:
com_df.to_csv('com_df.csv', index=False)